In [209]:
import requests
from bs4 import BeautifulSoup as soup
from requests_html import AsyncHTMLSession
import pandas as pd
import json
import re

In [210]:
header = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}


In [211]:
url = 'https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=microprocessor&ltype=wholesale&SortType=default&page=1'

In [212]:
html = requests.get(url,headers=header)


In [213]:
html.status_code


200

In [214]:
cookies = html.cookies

In [215]:
bsobj = soup(html.content, "html.parser")

In [216]:
next = bsobj.find(title='Next Page')
next

In [217]:
bsobj

<!DOCTYPE html>

<html lang="en" xmlns:th="http://www.w3.org/1999/xhtml">
<head>
<meta charset="utf-8"/>
<title>microprocessor - Buy microprocessor with free shipping on AliExpress</title>
<meta content="microprocessor" name="keywords">
<meta content="Quality microprocessor with free worldwide shipping on AliExpress" name="description"/>
<meta content="follow,index" name="robots"/>
<link href="https://www.aliexpress.com/w/wholesale-microprocessor.html" rel="canonical"/>
<link href="https://www.aliexpress.com/w/wholesale-microprocessor.html" hreflang="en" rel="alternate">
<link href="https://id.aliexpress.com/w/wholesale-microprocessor.html" hreflang="id" rel="alternate"/>
<link href="https://ko.aliexpress.com/w/wholesale-microprocessor.html" hreflang="ko" rel="alternate"/>
<link href="https://ar.aliexpress.com/w/wholesale-microprocessor.html" hreflang="ar" rel="alternate"/>
<link href="https://de.aliexpress.com/w/wholesale-microprocessor.html" hreflang="de" rel="alternate"/>
<link href

In [218]:
page = 1
moving = url[:-1] 
links = []
while html.status_code == 200:
    print(moving + str(page))
    productlist = bsobj.findAll('script',type="text/javascript")
    products = productlist[3]
    li = json.loads(re.findall(r"window.runParams\s*=\s*({.+?});", str(products))[0])
    
    if "mods" in li:
        for i in li["mods"]["itemList"]["content"]:
            links.append(i["productId"])
    else:
        break
    page += 1
    html = requests.get(moving + str(page),headers=header)
    cookies = html.cookies
    bsobj = soup(html.content, "html.parser")

https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=microprocessor&ltype=wholesale&SortType=default&page=1
https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=microprocessor&ltype=wholesale&SortType=default&page=2
https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=microprocessor&ltype=wholesale&SortType=default&page=3
https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=microprocessor&ltype=wholesale&SortType=default&page=4
https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=microprocessor&ltype=wholesale&SortType=default&page=5
https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=microprocessor&ltype=wholesale&SortType=default&page=6
https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=microprocessor&ltype=wholesale&SortType=default&page=7
https://www.aliexpress.com/wholesale?trafficChannel=mai

In [219]:
asession = AsyncHTMLSession()
r = await asession.get('https://www.aliexpress.com')
for f in links:
    print('https://www.aliexpress.com/item/' + f + ".html")

https://www.aliexpress.com/item/2255800243315514.html
https://www.aliexpress.com/item/3256803146302318.html
https://www.aliexpress.com/item/2251832701321414.html
https://www.aliexpress.com/item/3256803846715067.html
https://www.aliexpress.com/item/3256803850203105.html
https://www.aliexpress.com/item/3256803591005661.html
https://www.aliexpress.com/item/3256801814128802.html
https://www.aliexpress.com/item/3256803263737228.html
https://www.aliexpress.com/item/2251832698042672.html
https://www.aliexpress.com/item/3256803371028970.html
https://www.aliexpress.com/item/3256804336157932.html
https://www.aliexpress.com/item/3256803164782884.html
https://www.aliexpress.com/item/2255800363220505.html
https://www.aliexpress.com/item/3256803932178243.html
https://www.aliexpress.com/item/3256804056502496.html
https://www.aliexpress.com/item/3256803701068138.html
https://www.aliexpress.com/item/3256803413504396.html
https://www.aliexpress.com/item/3256802842359444.html
https://www.aliexpress.com/i

In [220]:
df = pd.DataFrame(columns=['Product Id','In Stock','Rating','Shipping Time','Shipping Location'])

In [221]:
for f in links:
    
    
    temp = {}
    try:
        r = await asession.get('https://www.aliexpress.com/item/' + f + ".html")
        print('https://www.aliexpress.com/item/' + f + ".html")
        await r.html.arender()
        h1 = r.html.find('h1')
        temp['Product Id'] = h1[0].text
        stock = r.html.find('.product-quantity-info')
        if('\n' in stock[0].text):
            temp['In Stock'] = stock[0].text.split("\n")[1]
        else:
            temp['In Stock'] = stock[0].text
        rating = r.html.find('.overview-rating-average')
        if(rating):
            temp['Rating'] = rating[0].text
        else:
            temp['Rating'] = "None"
        rating_count = r.html.find('.dynamic-shipping-line.dynamic-shipping-contentLayout')
        if("Estimated" in rating_count[0].text.strip()):
            temp['Shipping Time'] = rating_count[0].text.strip()
            temp['Shipping Location'] = rating_count[1].text.strip()
        else:
            temp['Shipping Time'] = rating_count[1].text.strip()
            temp['Shipping Location'] = rating_count[0].text.strip()
        df = df.append(temp, ignore_index=True)
    except:
        print("failure to process item, proceeding to the next")


https://www.aliexpress.com/item/2255800243315514.html
{'Product Id': '1pcs/lot MC6803P MC6803 MC6803CP DIP-40 Microcontroller/Microprocessor (MCU/MPU)', 'In Stock': '9997 Pieces available', 'Rating': 'None', 'Shipping Time': 'Estimated delivery on Oct 17', 'Shipping Location': 'From China to United States via Cainiao Super Economy Global'}
https://www.aliexpress.com/item/3256803146302318.html
{'Product Id': '1PCS New Original R6502AP R6502 6502AP DIP40 Microprocessor Chip', 'In Stock': '9728 Pieces available', 'Rating': '5.0', 'Shipping Time': 'Estimated delivery on Oct 17', 'Shipping Location': 'From China to United States via Cainiao Super Economy Global'}
https://www.aliexpress.com/item/2251832701321414.html
{'Product Id': 'MC68008P10 MC68008P MC68008 DIP48 16-Bit Microprocessor With 8-Bit Data Bus New original authentic 1pcs', 'In Stock': '1277 Pieces available', 'Rating': '5.0', 'Shipping Time': 'Estimated delivery on Oct 17', 'Shipping Location': 'From China to United States via 

RuntimeError: This event loop is already running

Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.detachFromTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.detachFromTarget: Target closed.
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.sendMessageToTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.sendMessageToTarget: Target closed.


: 

: 

In [ ]:
display(df)

,Product Id,In Stock,Rating,Shipping Time,Shipping Location
